In [16]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta
import math
import re
import pyodbc
import configparser


config = configparser.RawConfigParser()
config.read('config.ini')

server = config.get('DB-DETAILS','server')
database = config.get('DB-DETAILS','database')
username = config.get('DB-DETAILS','username')
password = config.get('DB-DETAILS','password')

lookup_high_risk = config.get('LOOKUP-TABLE-DETAILS','lookup_high_risk')
lookup_high_cost = config.get('LOOKUP-TABLE-DETAILS','lookup_high_cost')

auth_src_table = config.get('AUTH-TABLE-DETAILS','src_table')
auth_dest_table = config.get('AUTH-TABLE-DETAILS','dest_table')


cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

#cursor.execute("SELECT TOP(1) * FROM Canopy.ARCH_ADT_ANDOR_ED")

qr1 = "Select * from " + auth_src_table

qr2_High_Risk = "Select * from " + lookup_high_risk

qr3_High_Cost = "Select * from " + lookup_high_cost

Auth_data = pd.read_sql(qr1, cnxn)

#print(ADT_data)

df = pd.read_sql(qr2_High_Risk,cnxn)

df = df.drop_duplicates(keep='first')

high_cost_codes = pd.read_sql(qr3_High_Cost,cnxn)

Auth_data["DX1"] = Auth_data["DX1"].replace(",",";").apply(str)

Auth_data["DX1"].append(Auth_data["DX2"]).reset_index(drop=True)
Auth_data["DX_combined"] = Auth_data["DX1"].apply(str) +";" + Auth_data["DX2"].apply(str)+";" + Auth_data["DX3"].apply(str)+";" + Auth_data["DX4"].apply(str)+";" + Auth_data["DX5"].apply(str)+";" + Auth_data["DX6"].apply(str)+";" + Auth_data["DX7"].apply(str)+";" + Auth_data["DX8"].apply(str)+";" + Auth_data["DX9"].apply(str)



# filling 'blank_code' in place of null values

Auth_data['DX_combined'].fillna('blank_code', inplace=True)

# making list of diagnosis, high-risk and high-cost code

diagnosis_lst = Auth_data['DX_combined'].tolist()
#print(diagnosis_lst)
canopy_high_risk_codes = df['code'].tolist()

canopy_high_risk_prim_d = df['Primary Disease'].tolist()

high_cost_chronic_codes = high_cost_codes['code'].tolist()

high_cost_chronic_subcategory = high_cost_codes['Subcategory'].tolist()

text_code = ['abdominal pain','chest pain','back pain','breath','SOB','short of breath','fall','pregnancy','covid']

for i in range(len(canopy_high_risk_codes)):
    if(isinstance(canopy_high_risk_codes[i],int)):
        canopy_high_risk_codes[i]
    elif (pd.isnull(canopy_high_risk_codes[i])):
        None
    else:
        canopy_high_risk_codes[i] = canopy_high_risk_codes[i].lower()

        
for i in range(len(high_cost_chronic_codes)):
    if(isinstance(high_cost_chronic_codes[i],int)):
        high_cost_chronic_codes[i]
    elif(pd.isnull(high_cost_chronic_codes[i])):
        None
    else:
        high_cost_chronic_codes[i] = high_cost_chronic_codes[i].lower()
        

canopy_high_risk_codes = canopy_high_risk_codes + text_code
high_cost_chronic_codes = high_cost_chronic_codes + text_code


def high_risk_cost_flag (diagnosis_lst,canopy_high_risk_codes):
    b = []
    prim_d = []
    codes = []
    for i in range(0,len(diagnosis_lst)):
        a = diagnosis_lst[i]
        #print(a)
        split1 =  a.split(";")
        split = [item for item in split1 if not(item == 'nan')]
        #print(split)
        clean_lst = []
        
        for i in range(0,len(split)):
            a = split[i].strip()
            a = a.replace(".",'')
            a = a.lower()
            
            if(re.findall(r"abd*",a)):
                a = "abdominal pain"
            elif(re.findall(r"chest*",a)):
                a = "chest pain"
            elif(re.findall(r"back*",a)):
                a = "back pain"
            elif(re.findall(r"breath*",a) or re.findall(r"sob*",a)):
                a = "short of breath"
            elif(re.findall(r"fall*",a)):
                a = "fall"
            elif(re.findall(r"pregnancy*",a)):
                a = "pregnancy"
            elif(re.findall(r"covid*",a)):
                a = "covid"
            else:
                a
            clean_lst.append(a)

        if set(clean_lst).intersection(canopy_high_risk_codes):
            res = "Y"
            b.append(res)
        else:
            res = "N"
            b.append(res)
        
    return b

def high_risk_cost_desc (diagnosis_lst,canopy_high_risk_codes,col_name):
    b = []
    prim_d = []
    codes = []
    
    for i in range(0,len(diagnosis_lst)):
        a = diagnosis_lst[i]
        #split =  a.split(";")
        split1 =  a.split(";")
        split = [item for item in split1 if not(item == 'nan')]

        #print(split)
        clean_lst = []
        for i in range(0,len(split)):
            a = split[i].strip()
            a = a.replace(".",'')
            a = a.lower()
            if(re.findall(r"abd*",a)):
                a = "abdominal pain"
            elif(re.findall(r"chest*",a)):
                a = "chest pain"
            elif(re.findall(r"back*",a)):
                a = "back pain"
            elif(re.findall(r"breath*",a) or re.findall(r"sob*",a)):
                a = "short of breath"
            elif(re.findall(r"fall*",a)):
                a = "fall"
            elif(re.findall(r"pregnancy*",a)):
                a = "pregnancy"
            elif(re.findall(r"covid*",a)):
                a = "covid"
            else:
                a
            clean_lst.append(a)
        
        if set(clean_lst).intersection(canopy_high_risk_codes):
            prim_d = set(clean_lst).intersection(canopy_high_risk_codes)
            prim_d = list(prim_d)
            for i in range(len(prim_d)):
                if(isinstance(prim_d[i],int)):
                    prim_d[i]
                else:
                    prim_d[i] = prim_d[i].upper()
            lsttostr_prim = ','.join([str(elem) for elem in prim_d])
            
            if(col_name == "Primary Disease"):
                res = df[col_name][df['code'].isin(prim_d)].tolist()
            else:
                res = high_cost_codes[col_name][high_cost_codes['code'].isin(prim_d)].tolist()
                
            lst_set = set(res)
            listToStr = ','.join([str(elem) for elem in lst_set])
            if(len(res) == 0):
                b.append(lsttostr_prim)
            else: 
                b.append(listToStr)
                
        else:
            res1 = col_name
            res1  =  "Not a High Risk" if(col_name == "Primary Disease") else "Not a High Cost"
            b.append(res1)
    return b


Auth_data["High_Risk_Flag"] = high_risk_cost_flag(diagnosis_lst,canopy_high_risk_codes)

Auth_data["High_Risk_Desc"] = high_risk_cost_desc(diagnosis_lst,canopy_high_risk_codes,"Primary Disease")

Auth_data["High_Cost_Flag"] = high_risk_cost_flag(diagnosis_lst,high_cost_chronic_codes)

Auth_data["High_Cost_Desc"] = high_risk_cost_desc(diagnosis_lst,high_cost_chronic_codes,"Subcategory")


qry_insert = "Insert into " + auth_dest_table + "(AuthNo,Status,ReferringPhysNPI,ReferringPhysName,AdmittingPhysNPI,AdmittingPhysName,PCPNPI,PCPName,FacNPI,FacName,MemID,FirstName,LastName,MI,DOB,HealthPlan,ReceiveDate,AuthActionDate,ExpireDate,LengthOfStay,POSCode,Mod1,DischargeDate,AdmissionDate,ReferralType,AdmissionSource,Notes,DX1,DX2,DX3,DX4,DX5,DX6,DX7,DX8,DX9,FileName,FileTransferDate,Proc1,Proc2,Proc3,Proc4,Proc5,Proc6,Proc7,Proc8,Proc9,Proc10,RevCode,RevCodeLine,RevFromDate,RevThroughDate,RevStatus,Category,PhysicianGroup,CanopyUploadTS,High_Risk_Flag,High_Risk_Desc,High_Cost_Flag,High_Cost_Desc) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"

def check_null(column_value):
    if(pd.isnull(column_value)):
        column_value = None
    else:
        column_value
    return column_value

cursor.execute("truncate table " + auth_dest_table)

insert_batch = []
rec = ""
counter = 0;
batchnum = 1;

data_len = len(Auth_data)

for index,row in Auth_data.iterrows():
    rec = (row['AuthNo'],row['Status'],row['ReferringPhysNPI'],row['ReferringPhysName'],row['AdmittingPhysNPI'],row['AdmittingPhysName'],row['PCPNPI'],row['PCPName'],row['FacNPI'],row['FacName'],row['MemID'],row['FirstName'],row['LastName'],row['MI'],row['DOB'],row['HealthPlan'],check_null(row['ReceiveDate']),check_null(row['AuthActionDate']),check_null(row['ExpireDate']),check_null(row['LengthOfStay']),row['POSCode'],row['Mod1'],check_null(row['DischargeDate']),check_null(row['AdmissionDate']),row['ReferralType'],row['AdmissionSource'],row['Notes'],row['DX1'],row['DX2'],row['DX3'],row['DX4'],row['DX5'],row['DX6'],row['DX7'],row['DX8'],row['DX9'],row['FileName'],check_null(row['FileTransferDate']),row['Proc1'],row['Proc2'],row['Proc3'],row['Proc4'],row['Proc5'],row['Proc6'],row['Proc7'],row['Proc8'],row['Proc9'],row['Proc10'],row['RevCode'],row['RevCodeLine'],check_null(row['RevFromDate']),check_null(row['RevThroughDate']),row['RevStatus'],row['Category'],row['PhysicianGroup'],row['CanopyUploadTS'],row['High_Risk_Flag'],row['High_Risk_Desc'],row['High_Cost_Flag'],row['High_Cost_Desc'])
    insert_batch.append(rec)
    counter = counter + 1;
    if(counter%100 == 0):
        cursor.executemany(qry_insert,insert_batch)
        cursor.commit()
        counter = 0
        data_len = data_len - len(insert_batch)
        insert_batch = []
        batchnum = batchnum + 1

if data_len != 0:
    cursor.executemany(qry_insert,insert_batch)  

print("Data Load Done")

cursor.commit()
cnxn.commit()
cursor.close()

1
2
Data Load Done
